In [ ]:
import os
import glob

def find_longest_rir(rir_dir, sample_rate=44000):
    """
    Find and load the longest RIR file from a directory of .npy RIR files
    
    Args:
        rir_dir (str): Path to directory containing .npy RIR files
        sample_rate (int): Sample rate for duration calculations (default: 44000)
    
    Returns:
        tuple: (rir_array, rir_path, length_samples, duration_seconds)
    
    Usage in Jupyter:
    >>> longest_rir, path, length, duration = find_longest_rir('/path/to/rir/directory')
    """
    
    # Find all .npy files in the directory
    rir_pattern = os.path.join(rir_dir, "**/*.npy")
    rir_files = glob.glob(rir_pattern, recursive=True)
    
    if not rir_files:
        raise ValueError(f"No .npy files found in directory: {rir_dir}")
    
    print(f"Found {len(rir_files)} .npy files in {rir_dir}")
    
    longest_rir = None
    longest_path = None
    max_length = 0
    
    print("\nScanning RIR files for lengths...")
    
    for i, rir_path in enumerate(rir_files):
        try:
            # Load the RIR
            rir = np.load(rir_path, allow_pickle=True)
            
            # Handle different array shapes
            rir = np.squeeze(rir)
            if rir.ndim > 1:
                rir = rir.flatten()
            
            current_length = len(rir)
            
            # Print progress every 10 files or for the longest found so far
            if (i + 1) % 10 == 0 or current_length > max_length:
                duration = current_length / sample_rate
                print(f"  [{i+1:3d}/{len(rir_files)}] {os.path.basename(rir_path)}: {current_length} samples ({duration:.3f}s)")
            
            # Check if this is the longest so far
            if current_length > max_length:
                max_length = current_length
                longest_rir = rir
                longest_path = rir_path
                
        except Exception as e:
            print(f"  Error loading {rir_path}: {e}")
            continue
    
    if longest_rir is None:
        raise ValueError("No valid RIR files could be loaded")
    
    duration_seconds = max_length / sample_rate
    
    print(f"\n" + "="*60)
    print(f"LONGEST RIR FOUND:")
    print(f"  Path: {longest_path}")
    print(f"  Length: {max_length} samples")
    print(f"  Duration: {duration_seconds:.3f} seconds")
    print(f"  Shape: {longest_rir.shape}")
    print(f"  Data type: {longest_rir.dtype}")
    print(f"  Min/Max values: {longest_rir.min():.6f} / {longest_rir.max():.6f}")
    print("="*60)
    
    return longest_rir, longest_path, max_length, duration_seconds

In [1]:
#!/usr/bin/env python3
"""
Script to inspect and play RIR (Room Impulse Response) .npy files in Jupyter notebook
"""

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio, display
from scipy import signal
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def apply_rir_convolution(audio_path, rir_path, rir_native_rate=44100,target_rate=44100):
    # Load audio
    audio, sr = librosa.load(audio_path, sr=None)
    print("Playing Original audio at target rate:", sr)
    audio = librosa.resample(audio, orig_sr=sr, target_sr=target_rate, res_type="kaiser_fast")
    display(Audio(audio, rate=target_rate), display_id="original_audio")

    original_length = len(audio)
    logger.info(f"Audio shape: {audio.shape}, Sample rate: {sr}")
    
    rir = np.squeeze(np.load(rir_path, allow_pickle=True))
    rir = librosa.resample(
        rir, 
        orig_sr=rir_native_rate, 
        target_sr=target_rate, 
        res_type="kaiser_fast"
    )
    print("Playing RIR (Room Impulse Response) at target rate:", target_rate)
    display(Audio(rir, rate=target_rate), display_id="rir_audio")
    
    if rir is not None:
        audio = audio.flatten()
        rir = rir.flatten()
        
        # if len(audio) >= len(rir):
        #     audio_convolved = signal.convolve(audio, rir, mode="same")
        # else:
        audio_convolved = signal.convolve(audio, rir, mode="full")
    else:
        logger.warning("RIR is empty, skipping convolution")
        audio_convolved = audio
        
    actlev = np.max(np.abs(audio_convolved))
    if actlev > 0.99:
        audio_convolved = (audio_convolved / actlev) * 0.98

    if abs(original_length - len(audio_convolved)) > 10:
        logger.warning(f"length mismatch: {original_length} vs {len(audio_convolved)}")
        
    # Adjust length to match original
    if original_length > len(audio_convolved):
        final_audio = np.pad(audio_convolved, (0, original_length - len(audio_convolved)))
    elif original_length < len(audio_convolved):
        final_audio = audio_convolved[:original_length]
    else:
        final_audio = audio_convolved
    
    print("Playing normalized convolved audio at target rate:", target_rate, "and length:", len(final_audio))
    display(Audio(final_audio, rate=target_rate), display_id="convolved_audio")

short_audio = "/mnt/Projects/Projects/AudioProcessing/resemble-enhance/data/fg/Eleven/en-AU-DeanNeural/en-AU-DeanNeural-0.mp3"
long_audio = "/mnt/Projects/Projects/AudioProcessing/resemble-enhance/data/fg/Eleven/en-IN-NatashaNeural/en-IN-NatashaNeural-237.mp3"
audio = apply_rir_convolution(short_audio,
                     "/mnt/Store05/Datasets/RoomImpulseResponse/ConcatenatedRIR_v2/Arni/IR_numClosed_3_numComb_336_mic_4_sweep_3.npy")

Playing Original audio at target rate: 44100


Playing RIR (Room Impulse Response) at target rate: 44100


length mismatch: 58752 vs 164591


Playing normalized convolved audio at target rate: 44100 and length: 58752
